In [2]:
import pandas as pd

df = pd.read_csv("/content/14_word_vectors_spacy_text_classification/Fake_Real_Data.csv")
print(df.shape)
df.head(5)

(9900, 2)


,Text,label
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake
1,U.S. conservative leader optimistic of common ...,Real
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real
3,Court Forces Ohio To Allow Millions Of Illega...,Fake
4,Democrats say Trump agrees to work on immigrat...,Real


In [3]:
df['label'].value_counts()

,count
label,
Fake,5000
Real,4900


In [4]:
df['label_num'] = df['label'].map({'Real':1,'Fake':0})
df.head(5)

,Text,label,label_num
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0
1,U.S. conservative leader optimistic of common ...,Real,1
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0
4,Democrats say Trump agrees to work on immigrat...,Real,1


In [8]:
!python -m spacy download en_core_web_lg
import spacy
nlp = spacy.load('en_core_web_lg')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 3.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [9]:
df['vector'] = df['Text'].apply(lambda x: nlp(x).vector)

In [11]:
df.head(5)

,Text,label,label_num,vector
0,Top Trump Surrogate BRUTALLY Stabs Him In The...,Fake,0,"[-0.6759837, 1.4263071, -2.318466, -0.451093, ..."
1,U.S. conservative leader optimistic of common ...,Real,1,"[-1.8355803, 1.3101058, -2.4919677, 1.0268308,..."
2,"Trump proposes U.S. tax overhaul, stirs concer...",Real,1,"[-1.9851209, 0.14389805, -2.4221718, 0.9133005..."
3,Court Forces Ohio To Allow Millions Of Illega...,Fake,0,"[-2.7812982, -0.16120885, -1.609772, 1.3624227..."
4,Democrats say Trump agrees to work on immigrat...,Real,1,"[-2.2010763, 0.9961637, -2.4088492, 1.128273, ..."


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df.vector.values,
    df.label_num,
    test_size=0.2,
    random_state=2022
)

In [12]:
import numpy as np

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [16]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report

scaler = MinMaxScaler()
scaled_train_embed = scaler.fit_transform(X_train_2d)
scaled_test_embed = scaler.transform(X_test_2d)


clf = MultinomialNB()
clf.fit(scaled_train_embed, y_train)

MultinomialNB()

In [17]:
y_pred = clf.predict(scaled_test_embed)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.94      0.95      1024
           1       0.94      0.95      0.94       956

    accuracy                           0.94      1980
   macro avg       0.94      0.94      0.94      1980
weighted avg       0.94      0.94      0.94      1980



In [18]:
X_test[:5]

array([array([-1.82071769e+00, -1.67886421e-01, -1.58924341e+00,  2.64720857e-01,
               4.32017612e+00,  1.12455487e-01,  3.74780208e-01,  3.96886873e+00,
               4.08361077e-01, -7.32511461e-01,  4.43810892e+00,  1.27185071e+00,
              -2.99832225e+00,  1.17190230e+00,  6.74004495e-01,  1.44744599e+00,
               7.53936172e-01, -7.98084065e-02, -9.22594607e-01, -1.31350839e+00,
               1.42230511e+00, -6.35938287e-01, -8.13103855e-01, -2.06643671e-01,
               1.84903011e-01, -1.25880575e+00, -2.30192685e+00, -1.06831539e+00,
              -1.13668211e-01,  1.57176137e+00,  3.48270983e-01, -3.23561579e-01,
              -3.72438282e-01, -1.78332794e+00, -2.84954762e+00, -7.77942121e-01,
              -1.03924608e+00,  1.05204225e+00,  5.72074473e-01,  5.81632815e-02,
              -3.15533951e-02, -1.88561808e-03, -1.24629103e-01,  1.99970260e-01,
              -1.28027451e+00,  1.12080586e+00, -6.46281958e-01, -2.03803205e+00,
              -5

In [19]:
y_pred[:5]

array([0, 0, 0, 1, 1])

In [20]:
y_test[:5]

,label_num
4024,0
5098,0
281,0
4883,1
5493,1


In [28]:
print(df.iloc[281])

Text          Five Facts About The Obama Economy That Trump...
label                                                     Fake
label_num                                                    0
vector       [-1.9874974, 0.46089822, -2.3723187, 0.6806237...
Name: 281, dtype: object


In [29]:
from  sklearn.neighbors import KNeighborsClassifier

#1. creating a KNN model object
clf = KNeighborsClassifier(n_neighbors = 5, metric = 'euclidean')

#2. fit with all_train_embeddings and y_train
clf.fit(X_train_2d, y_train)

#3. get the predictions for all_test_embeddings and store it in y_pred
y_pred = clf.predict(X_test_2d)

#4. print the classfication report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      1024
           1       0.99      0.99      0.99       956

    accuracy                           0.99      1980
   macro avg       0.99      0.99      0.99      1980
weighted avg       0.99      0.99      0.99      1980

